In [1]:
import os
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [2]:
df = pd.read_csv("../static_merged_data.csv")
x = df.drop(labels="status", axis=1)
y = df[["status"]]

def prepare_train_test_set(X, Y, num_classes):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=.2)
    train_labels = y_train.status.astype('category').cat.codes.to_numpy()
    test_labels = y_test.status.astype('category').cat.codes.to_numpy()
    X_train.to_numpy()
    X_test.to_numpy()
    return (X_train, X_test), (train_labels, test_labels)

In [3]:
(X_train, X_test), (train_labels, test_labels) = prepare_train_test_set(x,y,3)

In [4]:
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Input, InputLayer
import tensorflow as tf
from keras import Sequential
from keras.layers import Activation 
model = Sequential([
    InputLayer(input_shape=(4,), name="input-layer"),
    Dense(64, activation="relu"),
    Dense(64, activation="relu"),
    Dense(128, activation="relu"),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(16, activation="relu"),
    Dense(3, name="output_layer")
], name="best_acc_model")

model.summary()

Model: "best_acc_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 256)               33024     
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 16)                2064      
                                                                 
 output_layer (Dense)        (None, 3)              

In [5]:
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=(model.name + ".h5"),
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

history = model.fit(X_train, train_labels, epochs=80, validation_split=0.1)

Epoch 1/80
2168/2168 [==============================] - 7s 3ms/step - loss: 0.7349 - accuracy: 0.6314 - val_loss: 0.3778 - val_accuracy: 0.8426
Epoch 2/80
2168/2168 [==============================] - 6s 3ms/step - loss: 0.2576 - accuracy: 0.8912 - val_loss: 0.3434 - val_accuracy: 0.8486
Epoch 3/80
2168/2168 [==============================] - 5s 3ms/step - loss: 0.2121 - accuracy: 0.9111 - val_loss: 0.2339 - val_accuracy: 0.9036
Epoch 4/80
2168/2168 [==============================] - 6s 3ms/step - loss: 0.2018 - accuracy: 0.9146 - val_loss: 0.1666 - val_accuracy: 0.9294
Epoch 5/80
2168/2168 [==============================] - 6s 3ms/step - loss: 0.1932 - accuracy: 0.9185 - val_loss: 0.1471 - val_accuracy: 0.9385
Epoch 6/80
2168/2168 [==============================] - 5s 2ms/step - loss: 0.1875 - accuracy: 0.9198 - val_loss: 0.1965 - val_accuracy: 0.9205
Epoch 7/80
2168/2168 [==============================] - 5s 2ms/step - loss: 0.1816 - accuracy: 0.9240 - val_loss: 0.1521 - val_accuracy:

In [6]:
_, baseline_model_accuracy = model.evaluate(X_test,test_labels, verbose=0)
print("Baseline model accuracy: ", baseline_model_accuracy)

Baseline model accuracy:  0.9629476070404053


In [7]:
keras.models.save_model(model, "own_model.h5", include_optimizer=False)

# Fine-tune pre-trained model with pruning

In [8]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 128
epochs = 2
validation_split = 0.1

num_samples = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_samples/batch_size).astype(np.int32) * epochs

# Define model for pruning
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.80, begin_step=0, end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# prune_low_magnitude requires a recompile

model_for_pruning.compile(optimizer='adam', loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model_for_pruning.summary()

Model: "best_acc_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense (  (None, 64)               578       
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_dense_1  (None, 64)               8258      
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_2  (None, 128)              16514     
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_3  (None, 256)              65794     
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_4  (None, 128)           

# Train and evalute the model against baseline model

In [9]:
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries("log_pruning_results")
]

model_for_pruning.fit(X_train,train_labels, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=callbacks)

Epoch 1/2
542/542 [==============================] - 5s 4ms/step - loss: 2.2533 - accuracy: 0.7547 - val_loss: 0.4478 - val_accuracy: 0.8268
Epoch 2/2
542/542 [==============================] - 2s 4ms/step - loss: 0.8451 - accuracy: 0.6041 - val_loss: 0.7013 - val_accuracy: 0.7228


In [10]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(X_test, test_labels, verbose=0)

print("Baseline test accuracy: ", baseline_model_accuracy)
print("Pruned test accuracy: ", model_for_pruning_accuracy)

Baseline test accuracy:  0.9629476070404053
Pruned test accuracy:  0.7163985371589661


In [11]:
# strip_pruning removes every tf.Variable that pruning only needs during training, which would otherwise add to model size during inference
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

pruned_keras_file = "pruned_own_model.h5"
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

pruned_tflite_file = "pruned_tflite_own_model.tflite"

with open(pruned_tflite_file, "wb") as f:
    f.write(pruned_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp3vs4uzq_\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp3vs4uzq_\assets


In [13]:
import gzip
import os 
import zipfile
def get_gzipped_model_size(file, file_name):
    # Returns size of gzipped model, in bytes
    zipped_file = file_name + ".zip"

    with zipfile.ZipFile(zipped_file, "w", compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()
quantized_and_pruned_tflite_file = "quantized_pruned_own_model.tflite"

with open(quantized_and_pruned_tflite_file, "wb") as f:
    f.write(quantized_and_pruned_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp3asnp8hl\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmp3asnp8hl\assets


In [15]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size("own_model.h5", "zip_own_model")))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size("pruned_own_model.h5", "zip_pruned_own_model")))
print("Size of gzipped pruned TFLite model: %.2f bytes" % (get_gzipped_model_size("pruned_tflite_own_model.tflite", "zip_pruned_tflite_own_model")))
print("Size of gzipped quantized pruned TFLite model: %.2f bytes" % (get_gzipped_model_size("quantized_pruned_own_model.tflite", "zip_quantized_pruned_own_model")))

Size of gzipped baseline Keras model: 304157.00 bytes
Size of gzipped pruned Keras model: 100100.00 bytes
Size of gzipped pruned TFLite model: 98262.00 bytes
Size of gzipped quantized pruned TFLite model: 28106.00 bytes


In [16]:
import numpy as np

def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on over y image in the test dataset
    prediction_digits = []
    for i, test_image in enumerate(X_test.to_numpy()):
        if i % 1000 == 0:
            print("Evaluated on {n} results so far.".format(n=i))
        
        # Pre-processing: add batch dimension and convert to float32 to match with the model's input data format
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest probability
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    print("\n")

    # Compare prediction result with ground truth labels to calculate accuracy
    prediction_digits = np.array(prediction_digits)
    print(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy

In [17]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print("Pruned and quantized TFLite test_accuracy: ", test_accuracy)
print("Pruned TF test accuracy: ", model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.
Evaluated on 10000 results so far.
Evaluated on 11000 results so far.
Evaluated on 12000 results so far.
Evaluated on 13000 results so far.
Evaluated on 14000 results so far.
Evaluated on 15000 results so far.
Evaluated on 16000 results so far.
Evaluated on 17000 results so far.
Evaluated on 18000 results so far.
Evaluated on 19000 results so far.


[1 2 0 ... 2 1 0]
Pruned and quantized TFLite test_accuracy:  0.700570835495589
Pruned TF test accuracy:  0.7163985371589661


In [152]:
def representative_dataset():
    for sample in X_train.to_numpy()[:500,:]:
      yield [sample.astype(np.float32)]

In [153]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpvb_cwjdp\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpvb_cwjdp\assets
c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [124]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model_int8 = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpfgza5fy0\assets


INFO:tensorflow:Assets written to: C:\Users\sandr\AppData\Local\Temp\tmpfgza5fy0\assets
c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [154]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()
# interpreter_int8 = tf.lite.Interpreter(model_content=tflite_quant_model_int8)
# interpreter_int8.allocate_tensors()
test_accuracy = evaluate_model(interpreter)
# test_accuracy_int8 = evaluate_model(interpreter_int8)


print("Pruned and quantized TFLite test_accuracy: ", test_accuracy)
# print("Int_8 Pruned and quantized TFLite test_accuracy: ", test_accuracy_int8)
print("Pruned TF test accuracy: ", model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.
Evaluated on 10000 results so far.
Evaluated on 11000 results so far.
Evaluated on 12000 results so far.
Evaluated on 13000 results so far.
Evaluated on 14000 results so far.
Evaluated on 15000 results so far.
Evaluated on 16000 results so far.
Evaluated on 17000 results so far.
Evaluated on 18000 results so far.
Evaluated on 19000 results so far.


Pruned and quantized TFLite test_accuracy:  0.6409963674104826
Pruned TF test accuracy:  0.9869226813316345
